In [3]:
import configparser

config = configparser.ConfigParser()
config.read('../../../data/config/config.cfg')

phenopackets_out = config.get('Paths', 'phenopackets_out')

In [4]:
print(phenopackets_out)

../../data/out/phenopackets/
